In [ ]:
"""
================================================================
        The Fermi-Pasta-Ulam-Tsingou (FPUT) Problem (1D)
================================================================
"""
from math import log10
import glob
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
import re
from scipy.interpolate import interp1d

#   Improve performance by applying 'jit' decorator
from numba import jit

__version__ = '1.0'


In [ ]:
datasets = {
    'displacements_1D_fixed': {},
    'displacements_1D_periodic': {},
    'velocities_1D_fixed': {},
    'velocities_1D_periodic': {}
}

all_dataset_filenames = glob.glob(f'./subset_data/*.txt')

for fn in all_dataset_filenames:
    matched = re.match(r'./subset_data/([a-z]+_1D_[a-z]+)_(\d+)_([\dd]+)_([\dd]+)_(\d+)_([\dd]+)\((-?\d+)\)', fn, re.IGNORECASE)

    if matched:
        loaded_data = np.loadtxt(fn)

        class_type, N, ALPHA, BETA, t_max, dt_mantissa, dt_exp = matched.groups()

        ALPHA = float(ALPHA.replace('d', '.'))
        BETA = float(BETA.replace('d', '.'))
        t_max = 10 ** int(t_max)
        dt = float(dt_mantissa.replace('d', '.')) * 10 ** int(dt_exp)
        config_str = f"{int(N)}_{str(ALPHA).replace('.', "d")}_{str(BETA).replace('.', "d")}_{int(log10(t_max))}_{str(dt_mantissa).replace('.', "d")}({int(dt_exp)})"

        num_steps = int(np.ceil(t_max / dt)) + 1        #   Number of time steps
        time_points = np.linspace(0, t_max + dt, num_steps + 1)

        datasets[class_type][fn] = {
            'count': N,
            'alpha': ALPHA,
            'beta': BETA,
            't_max': t_max,
            'time_step': dt,
            'time_points': time_points,
            'time_points_len': len(time_points),
            'data': loaded_data,
            'data_len': len(loaded_data)
        }
    else:
        print("Filename format does not match expected pattern. ")


In [ ]:
def find_smallest_time_points_len(dictionary):
    smallest_value = None
    
    def recursive_search(dictionary):
        nonlocal smallest_value
        
        for key, value in dictionary.items():
            if type(value) is dict:
                recursive_search(value)
            elif key == "time_points_len":
                if smallest_value is None or value < smallest_value:
                    smallest_value = value
    
    recursive_search(dictionary)
    
    return smallest_value


In [ ]:
def interpolate_dataset(original_time_points, data, common_time_points):
    interpolator = interp1d(original_time_points, data, kind='linear', fill_value="extrapolate")
    return interpolator(common_time_points)


In [ ]:
common_time_points = np.linspace(0, 1e4, find_smallest_time_points_len(datasets))

for category, category_data in datasets.items():
    for fn, details in category_data.items():
        original_time_points = details['time_points']
        data = details['data']

        interpolated_data = interpolate_dataset(original_time_points, data, common_time_points)
        
        datasets[category][fn]['interpolated_data'] = interpolated_data


In [ ]:
def calculate_rmse(data1, data2):
    mse = np.mean((data1 - data2) ** 2)
    rmse = np.sqrt(mse)
    return rmse

class_names = ["displacements_1D_fixed", "displacements_1D_periodic", "velocities_1D_fixed", "velocities_1D_periodic"]

errors = {}

for cn in class_names:
    reference_data = datasets[cn][f'./subset_data/{cn}_64_1d0_0d1_4_7d5(-4).txt']['interpolated_data']

    errors_cn = []

    for filename, details in datasets[cn].items():
        rmse_value = calculate_rmse(reference_data, details['interpolated_data'])
        errors_cn.append([details['time_step'], rmse_value])

    errors[cn] = np.array(errors_cn)

# Loop through the datasets dictionary
for (label, data), color in zip(errors.items(), ['red', 'blue', 'green', 'purple']):
    plt.loglog(data[:, 0], data[:, 1], c = color, label = label, marker = '.', ls = '', lw = '0.5')

plt.title(r'RMSE compared to $\Delta t = 7.5 \times 10^{-4}$')
plt.xlabel('Chosen time step')
plt.ylabel('RMSE')
plt.grid(True)
plt.legend()

plt.show()


We note two anomalies corresponding to time steps $\Delta t = 0.075$ and at $\Delta t = 7.5 \times 10^{-3}$. It is likely that this error arose from unexpected behaviors of `scipy.interpolate` 1D interpolation function. 